# Classification Dünnschnabelmöwen und Dickhornschafe

Author: Nils Bestehorn

Matrikelnummer: 1242890

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from mpl_toolkits.mplot3d import Axes3D #gehört zu matplotlib.pyplot
import seaborn as sns
#import mpl_toolkits.mplot3d

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#print(tf.__version__)

## Funktionen

In [ ]:
def plot_loss(history):
    plt.figure()
    
    plt.plot(history.history['loss'], label='Trainings Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    
    plt.title('Model Loss')
    plt.xlabel('Epoche')
    plt.ylabel('Loss (MSE)')
    
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_losses(history1,history2, model1_name="",model2_name=""):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Plot 1
    axes[0].plot(history1.history['loss'], label='Trainings Loss')
    axes[0].plot(history1.history['val_loss'], label='Validation Loss')

    axes[0].set_title(f'Training und Validierung {model1_name}')
    axes[0].set_xlabel('Epoche')
    axes[0].set_ylabel('Loss (Binary Crossentropy)')

    axes[0].legend()
    axes[0].grid(True)

    # Plot 2
    axes[1].plot(history2.history['loss'], label='Trainings Loss')
    axes[1].plot(history2.history['val_loss'], label='Validation Loss')

    axes[1].set_title(f'Training und Validierung {model2_name}')
    axes[1].set_xlabel('Epoche')
    axes[1].set_ylabel('Loss (Binary Crossentropy)')

    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

def plot_decision_background_from_df(model,df,ax,feature_cols,step=0.02,alpha=0.25):
    X = df[feature_cols].to_numpy() 

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, step),
        np.arange(y_min, y_max, step)
    )

    grid = np.c_[xx.ravel(), yy.ravel()]
    probs = model.predict(grid, verbose=0)
    Z = (probs > 0.5).astype(int).reshape(xx.shape)

    ax.contourf(
        xx, yy, Z,
        alpha=alpha,
        levels=[-0.5, 0.5, 1.5],
        colors=["#6fa8dc", "#e06666"]  # blau / rot
    )


def plot_decision_background(model,X,ax,step=0.02,padding=5,alpha=0.25):
    """plot_decision_background\n
        model=>DL Model\n
        X = df_source[["Groesse", "Umfang"]].to_numpy() 
    """
    # Bereich festlegen
    x_min, x_max = X[:, 0].min() - padding, X[:, 0].max() + padding
    y_min, y_max = X[:, 1].min() - padding, X[:, 1].max() + padding

    # Gitter erzeugen
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, step),
        np.arange(y_min, y_max, step)
    )

    grid = np.c_[xx.ravel(), yy.ravel()]
    probs = model.predict(grid, verbose=0)
    Z = (probs > 0.5).astype(int).reshape(xx.shape)

    ax.contourf(
        xx, yy, Z,
        alpha=alpha,
        levels=[-0.5, 0.5, 1.5],
        colors=["#6fa8dc", "#e06666"]  # blau / rot
    )

def plot_classes_2_models(model1,model2, model1_name="",model2_name=""):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Plot 1
    axes[0].plot(history1.history['loss'], label='Trainings Loss')
    axes[0].plot(history1.history['val_loss'], label='Validation Loss')

    axes[0].set_title(f'Training und Validierung {model1_name}')
    axes[0].set_xlabel('Epoche')
    axes[0].set_ylabel('Loss (Binary Crossentropy)')

    axes[0].legend()
    axes[0].grid(True)

    # Plot 2
    axes[1].plot(history2.history['loss'], label='Trainings Loss')
    axes[1].plot(history2.history['val_loss'], label='Validation Loss')

    axes[1].set_title(f'Training und Validierung {model2_name}')
    axes[1].set_xlabel('Epoche')
    axes[1].set_ylabel('Loss (Binary Crossentropy)')

    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()



## More Setup

Daten Laden und eine Übersicht über die Daten verschaffen

In [ ]:
notebooks_folder = os.getcwd()
projekt_folder = os.path.dirname(notebooks_folder)
source_data=os.path.join(projekt_folder,"data","animals.csv")

df_source=pd.read_csv(source_data)
df_source.info()
#print(df_source.info())
print()
print("Nan Check:")
print(df_source.isna().sum())
print()
print("Übersicht Über alle Daten:")
print(df_source.describe())

In [ ]:
df_source.head(5)
#print(df_source.head(5))

Label Typen Testen

In [ ]:
label_types=df_source['Label'].unique()
print(label_types)

Label 1 (Duennschnabelmoewe) Übersicht

In [ ]:
df_source[df_source['Label']==label_types[0]].describe()

Label 2 (Dickhornschaf) Übersicht

In [ ]:
df_source[df_source['Label']==label_types[1]].describe()

Übersichts Plot

In [ ]:
# Boolesche Masken
mask_schaf = df_source["Label"] == "Dickhornschaf"
mask_moewe = df_source["Label"] == "Duennschnabelmoewe"

plt.figure(figsize=(8, 6))
plt.scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
plt.scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)
plt.xlabel("Größe")
plt.ylabel("Umfang")
plt.title("Tiere nach Größe und Umfang")
plt.legend()
plt.margins(x=0.1, y=0.1)
plt.grid(False)
plt.show()



#plt.figure(figsize=(8, 6))
#sns.scatterplot(
#    data=df_source,
#    x="Groesse",
#    y="Umfang",
#    hue="Label",
#    style="Label",
#    alpha=0.7
#)
#plt.title("Tiere nach Größe und Umfang")
#plt.xlabel("Größe")
#plt.ylabel("Umfang")
#plt.grid(True)
#plt.margins(x=0.1, y=0.1)
#plt.tight_layout()
#plt.show()

# Preprocess the Data
Kodiere die Labels als numerische Werte, damit sie von
Tensorflow verarbeitet werden können. Erstellung einer classen map.

In [ ]:
class_name = {label: idx for idx, label in enumerate(label_types)}
print(class_name)

In [ ]:
#df_source=df_source.rename(columns={"Label":"Tiername"})
df_source["Label_encoded"] = df_source["Label"].map(class_name)
assert df_source["Label_encoded"].isna().sum() == 0
df_source


In [ ]:
# One-Hot hier nicht nötig!!!
df_source_test=df_source.copy()
df_source_test["Duennschnabelmoewe"] = (
    df_source_test["Label"] == "Duennschnabelmoewe"
).astype(int)

df_source_test["Dickhornschaf"] = (
    df_source_test["Label"] == "Dickhornschaf"
).astype(int)
df_source_test

Unnötigen Speicher freigeben!

In [ ]:
import gc

del df_source_test

gc.collect()

## Splitting Data
80% Training, 20% Test

In [ ]:
# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
df = df_source.copy()
df.pop('Label') #Label vorher wegwerfen

train_dataset = df.sample(frac=0.8, random_state=SEED)
test_dataset = df.drop(train_dataset.index)

Setting Labels

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Label_encoded')
#train_labels_names = train_features.pop('Label')

test_labels = test_features.pop('Label_encoded')
#test_labels_names = test_features.pop('Label')

# Normalization

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1) #feature weise normelarisierung
normalizer.adapt(np.array(train_features))
print("Features Normalized!")
print(normalizer.mean.numpy())
print()

first = np.array(train_features[:1])
with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

#Speicher für alle Ergebnisse wenn mehrere Modelle getestet
np.array(normalizer(train_features)).mean()


# Build the Model
Erstelle in Keras ein Modell mit 2 Hidden Layers
mit jeweils 4 Knoten und Relu-Aktivierung. Der Output-Knoten soll die Sigmoid-
Funktion als Aktivierung haben. Nimm Binary Cross Entropy als Loss-Funktion, und den
Adam-Optimizer.

In [ ]:
def build_and_compile_model(norm=None):
  if norm:
    print("Model with Normalization selected")
    model = keras.Sequential([
        norm,
        layers.Dense(4, activation='relu'),
        layers.Dense(4, activation='relu'),
        layers.Dense(1, activation="sigmoid")
    ])
  else:
    print("Model without Normalization selected")
    model = keras.Sequential([
        layers.Dense(4, activation='relu'),
        layers.Dense(4, activation='relu'),
        layers.Dense(1, activation="sigmoid")
    ])

    #Calls sind hier gleich!
  model.compile(#loss='binarycrossentropy',
                loss=keras.losses.BinaryCrossentropy(from_logits=True),
                #optimizer='adam',
                metrics=["accuracy"],
                optimizer=keras.optimizers.Adam(0.001)
                )
  return model

In [ ]:
test_results = {}
dnn_model = build_and_compile_model()
dnn_model_large_epochs = build_and_compile_model()
dnn_model_norm = build_and_compile_model(normalizer)
dnn_model_norm_large_epochs = build_and_compile_model(normalizer)


## Train the Models
Trainiere das Modell 50 (eventuell mehr) Epochen lang, mit einer
Batch-Größe von 100

### Base Model (dnn_model)

In [ ]:
history_dnn_model = dnn_model.fit(
    train_features,
    train_labels,
    epochs=50,
    batch_size=100,
    validation_split=0.2,
    verbose=0)


In [ ]:
dnn_model.summary()
test_results['dnn_model'] = dnn_model.evaluate(
    test_features, test_labels,
    verbose=0)

### Base Model Large Epochs (dnn_model_large_epochs)

In [ ]:
history_dnn_model_large_epochs = dnn_model_large_epochs.fit(
    train_features,
    train_labels,
    epochs=500,
    batch_size=100,
    validation_split=0.2,
    verbose=0)

In [ ]:
dnn_model_large_epochs.summary()
test_results['dnn_model_large_epochs'] = dnn_model_large_epochs.evaluate(
    test_features, test_labels,
    verbose=0)

### Base Model Normalized (dnn_model_norm)

In [ ]:
history_dnn_model_norm = dnn_model_norm.fit(
    train_features,
    train_labels,
    epochs=50,
    batch_size=100,
    validation_split=0.2,
    verbose=0)

In [ ]:
dnn_model_norm.summary()
test_results['dnn_model_norm'] = dnn_model_norm.evaluate(
    test_features, test_labels,
    verbose=0)

### Base Model Normalized Large Epochs

In [ ]:
history_dnn_model_norm_large_epochs = dnn_model_norm_large_epochs.fit(
    train_features,
    train_labels,
    epochs=500,
    batch_size=100,
    validation_split=0.2,
    verbose=0)

In [ ]:
dnn_model_norm_large_epochs.summary()
test_results['dnn_model_norm_large_epochs'] = dnn_model_norm_large_epochs.evaluate(
    test_features, test_labels,
    verbose=0)

# Validate the Models

In [ ]:
#print(test_results)
results_df=pd.DataFrame(test_results, index=['loss','accuracy']).T
results_df

In [ ]:
results_df.plot(kind="bar", figsize=(8,5))
plt.title("Modellvergleich auf dem Testset")
plt.show()


In [ ]:
plot_losses(history_dnn_model,history_dnn_model_large_epochs,model1_name="dnn_model",model2_name="dnn_model_large_epochs")
plot_losses(history_dnn_model_norm,history_dnn_model_norm_large_epochs,model1_name="dnn_model_norm",model2_name="dnn_model_norm_large_epochs")

Plotte die Loss-Funktion über Training und Test. 
Wie unterscheidet sich der Verlauf für wenige und viele Epochen?

- Die Modelle mit Wenig Epochen sind noch weiter weg zu den Trainings Kurven wärend durch mehr Epochen sich die Validation dem Training annähert!

In [ ]:
# Bereich festlegen
x_min, x_max = df_source["Groesse"].min() - 5, df_source["Groesse"].max() + 5
y_min, y_max = df_source["Umfang"].min() - 5, df_source["Umfang"].max() + 5

# Gitter erzeugen
xx, yy = np.meshgrid(
    np.linspace(x_min, x_max, 300),
    np.linspace(y_min, y_max, 300)
)

grid = np.c_[xx.ravel(), yy.ravel()]

pred_probs = dnn_model_norm_large_epochs.predict(grid, verbose=0)
pred_classes = (pred_probs > 0.5).astype(int)
X = pred_classes.reshape(xx.shape)

In [ ]:
X = df_source[["Groesse", "Umfang"]].to_numpy()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_decision_background(dnn_model, X, axes[0],step=0.1, padding=10)
axes[0].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[0].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)
axes[0].set_xlabel("Größe")
axes[0].set_ylabel("Umfang")
axes[0].set_title("Modell-Entscheidungsfläche (dnn_model)")
axes[0].legend()
plot_decision_background(dnn_model_large_epochs, X, axes[1],step=0.1, padding=10)
axes[1].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[1].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)
axes[1].set_xlabel("Größe")
axes[1].set_ylabel("Umfang")
axes[1].set_title("Modell-Entscheidungsfläche (dnn_model_large_epochs)")
axes[1].legend()
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_decision_background(dnn_model_norm, X, axes[0],step=0.1, padding=10)
axes[0].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[0].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)
axes[0].set_xlabel("Größe")
axes[0].set_ylabel("Umfang")
axes[0].set_title("Modell-Entscheidungsfläche (dnn_model_norm)")
axes[0].legend()
plot_decision_background(dnn_model_norm_large_epochs, X, axes[1],step=0.1, padding=10)
axes[1].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[1].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)
axes[1].set_xlabel("Größe")
axes[1].set_ylabel("Umfang")
axes[1].set_title("Modell-Entscheidungsfläche (dnn_model_norm_large_epochs)")
axes[1].legend()
plt.show()


## Prediction on new Data
Berechne zuletzt die Vorhersagewerte des Modells für folgende Wertepaare:
Umfang/Groesse: [90, 90], [70, 70]

In [ ]:
daten = [
    [90, 90],
    [70, 70]
]

prediction_data = pd.DataFrame(daten, columns=['Umfang', 'Groesse'])
prediction_data

In [ ]:
def predict_to_df(models, X, threshold=0.5):
    """
    models: dict {name: keras_model}
    X: DataFrame oder numpy array
    """
    df = X.copy()

    for name, model in models.items():
        probs = model.predict(X, verbose=0).ravel()
        df[f"{name}_prob"] = probs
        df[f"{name}_class"] = (probs > threshold).astype(int)

    return df
models = {
    "base": dnn_model,
    "many_epochs": dnn_model_large_epochs,
    "norm": dnn_model_norm,
    "norm_many_epochs": dnn_model_norm_large_epochs
}

In [ ]:
results_df = predict_to_df(models, prediction_data)
results_df

In [ ]:
results_df.filter(like="_prob").style.background_gradient(
    cmap="coolwarm"
)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_decision_background(dnn_model, X, axes[0],step=0.1, padding=20)
axes[0].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[0].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)

axes[0].scatter(
    results_df["Groesse"],
    results_df["Umfang"],
    facecolors="white",
    edgecolors="black",
    label="Prediction Data"
)

axes[0].set_xlabel("Größe")
axes[0].set_ylabel("Umfang")
axes[0].set_title("Modell-Entscheidungsfläche (dnn_model)")
axes[0].legend()
plot_decision_background(dnn_model_large_epochs, X, axes[1],step=0.1, padding=20)
axes[1].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[1].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)
axes[1].scatter(
    results_df["Groesse"],
    results_df["Umfang"],
    facecolors="white",
    edgecolors="black",
    label="Prediction Data"
)

axes[1].set_xlabel("Größe")
axes[1].set_ylabel("Umfang")
axes[1].set_title("Modell-Entscheidungsfläche (dnn_model_large_epochs)")
axes[1].legend()
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_decision_background(dnn_model_norm, X, axes[0],step=0.1, padding=20)
axes[0].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[0].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)
axes[0].scatter(
    results_df["Groesse"],
    results_df["Umfang"],
    facecolors="white",
    edgecolors="black",
    label="Prediction Data"
)

axes[0].set_xlabel("Größe")
axes[0].set_ylabel("Umfang")
axes[0].set_title("Modell-Entscheidungsfläche (dnn_model_norm)")
axes[0].legend()
plot_decision_background(dnn_model_norm_large_epochs, X, axes[1],step=0.1, padding=20)
axes[1].scatter(
    df_source.loc[mask_schaf, "Groesse"],
    df_source.loc[mask_schaf, "Umfang"],
    color="red",
    label="Dickhornschaf",
    edgecolor="k"
)
axes[1].scatter(
    df_source.loc[mask_moewe, "Groesse"],
    df_source.loc[mask_moewe, "Umfang"],
    color="blue",
    label="Dünnschnabelmöwe",
    edgecolor="k"
)

axes[1].scatter(
    results_df["Groesse"],
    results_df["Umfang"],
    facecolors="white",
    edgecolors="black",
    label="Prediction Data"
)
axes[1].set_xlabel("Größe")
axes[1].set_ylabel("Umfang")
axes[1].set_title("Modell-Entscheidungsfläche (dnn_model_norm_large_epochs)")
axes[1].legend()
plt.show()